# Real Time Eye Gaze Tracking with Kinect


In [164]:
from PIL import Image
import cv2
import numpy as np
import os
import json
import pandas as pd
import bson
from tqdm import tqdm
from pprint import pprint
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline

import sys
sys.path.append('../')

from brspy.export import GazesExport
from brspy.export import MimicsExport
from brspy.export import JointsExport
from brspy.export import JointOrientationsExport
from brspy.export import FacePropertiesExport

from brspy.reader import Session
from brspy.export.utils import *

from devices import Camera
from devices import Device

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

## Configuration

In [165]:
markers_path = r'./markers.json'
datasets_path = r"./datasets/07_08_18"
sess_path = os.path.join(datasets_path, '1533638926')
cam_data_path = r'./cam_data.json'

with open(markers_path, 'r') as f:
    markers_db = json.load(f)
    
with open(cam_data_path, 'r') as f:
    cam_data = json.load(f)
    
class HDFace:
    
    LefteyeInnercorner = '210'
    LefteyeOutercorner = '469'
    LefteyeMidtop = '241'
    LefteyeMidbottom = '1104'
    RighteyeInnercorner = '843'
    RighteyeOutercorner = '1117'
    RighteyeMidtop = '731'
    RighteyeMidbottom = '1090'
    LefteyebrowInner = '346'
    LefteyebrowOuter = '140'
    LefteyebrowCenter = '222'
    RighteyebrowInner = '803'
    RighteyebrowOuter = '758'
    RighteyebrowCenter = '849'
    MouthLeftcorner = '91'
    MouthRightcorner = '687'
    MouthUpperlipMidtop = '19'
    MouthUpperlipMidbottom = '1072'
    MouthLowerlipMidtop = '10'
    MouthLowerlipMidbottom = '8'
    NoseTip = '18'
    NoseBottom = '14'
    NoseBottomleft = '156'
    NoseBottomright = '783'
    NoseTop = '24'
    NoseTopleft = '151'
    NoseTopright = '772'
    ForeheadCenter = '28'
    LeftcheekCenter = '412'
    RightcheekCenter = '933'
    Leftcheekbone = '458'
    Rightcheekbone = '674'
    ChinCenter = '4'
    LowerjawLeftend = '1307'
    LowerjawRightend = '1327'
    
    @classmethod
    def items(cls):
        return {attr_name: getattr(cls, attr_name)
                for attr_name in dir(cls) if not attr_name.startswith('__') and not callable(getattr(cls, attr_name))}
    
    @classmethod
    def values(cls):
        return [value for key, value in cls.items().items()]
    
    @classmethod
    def keys(cls):
        return [key for key, value in cls.items().items()]
    
EyeRightCircleIdx = [776,  777,  846,  843,  1098, 
                     1097,  1095,  1096,  1096,  1091, 
                     1090,  1092,  1099,  1094,  1093, 
                     1100,  1101,  1102,  1117,  1071, 
                     1012,  992,  987,  752,  749, 
                     876,  733,  731,  728]
EyeLeftCircleIdx = [210, 316, 187, 153, 121,
                    241, 244, 238, 137, 211,
                    188, 287, 440, 1116, 469,
                    1115, 1114, 1113, 1107, 1106,
                    1112, 1105, 1104, 1103, 1108,
                    1109, 1111, 1110]

hd_all = list(map(str, range(0, 1347)))

avg_eye_radius = 0.007462746808244293

### Utils

In [166]:
from scipy.optimize import minimize


class Intersection:
    
    def __init__(self, cam_point, eye_radius, alpha=0.0001, tol=None):
        
        # model parameters
        self.eye_radius = eye_radius
        self.alpha = alpha
        self.tol = tol
        
        # points in 3d
        self.cam_point = cam_point
        self.eye_center = None
        self.ray_point = None
        
        
    def cost(self, point):
        
        # point must be on sphere surface
        sphere_cost = np.abs(np.sum((point - self.eye_center) ** 2) - (self.eye_radius ** 2))
        
        # point must be on line
        line_eq_components = (point - self.cam_point) / (self.ray_point - self.cam_point)
#         print(sphere_cost)
        line_cost = np.sum(np.abs(np.diff((point - self.cam_point) / (self.ray_point - self.cam_point))))
        
        # to prevent 2 point intersection ambiguity
        regularization = np.linalg.norm(point)
        
        # sum of all costs
        return sphere_cost ** 2 + line_cost ** 2 + sphere_cost * line_cost + self.alpha * regularization
    
    def find(self, eye_center, ray_point, verbose=True):
        
        # assign points
        self.eye_center = eye_center
        self.ray_point = ray_point
        
        # try minimize
        result = minimize(self.cost, x0=self.eye_center - self.ray_point, tol=self.tol)
        
        # print message
        if verbose:
            print(f'Cost: {result.fun:.2e}. ', result.message, sep='')
            
        return result.x
    
    def np_find(self, arr):
        return self.find(eye_center=arr[:3], ray_point=arr[3:], verbose=False)
    
    
def add_dummy_z(arr_2d):
    return np.column_stack((arr_2d, np.full(arr_2d.shape[0], 1))) 

def to_tuple(dct):
    return tuple(int(item) for key, item in dct.items())

def extract_rectangle(img, pt0, pt1):
    return img[pt0[1]:pt1[1], pt0[0]:pt1[0]]

def dict_to_np(dict_data):
    return np.array([value for axis, value in dict_data.items()]).reshape(-1, len(dict_data))

def add_dummy_z(arr_2d):
    if isinstance(arr_2d, dict):
        arr_2d = dict_to_np(arr_2d)
    return np.column_stack((arr_2d, np.full(arr_2d.shape[0], 1))) 

### Cameras 

In [167]:
for cam_name, data_dict in cam_data.items():
    Camera(cam_name, **data_dict)
    
KinectColor = Camera.get('KinectColor')
WebCamera = Camera.get('WebCamera')
InfraredCamera = Camera.get('InfraredCamera')
KinectInfrared = Camera.get('KinectInfrared')

### write markers to session

In [ ]:
def to_json(vector):
    return {axis: value for axis, value in zip('XYZ', vector)}

# choose only wall 0
markers_wall_0 = {key: value for key, value in markers_db.items() if key.split('_')[1] == '0'}
pprint(markers_wall_0)

markers_save = os.path.join(sess_path, 'DataSource', 'cam_101')

try:
    os.mkdir(markers_save)
except:
    pass

counter = 0
for i in range(5):
    for j in range(8):
        marker = markers_wall_0.get(f'wall_{0}_dot_{j+1}')
        if marker:
            file_name = os.path.join(markers_save, str(counter).rjust(5, '0') + '.txt')
            with open(file_name, 'w') as f:
                json.dump(to_json(marker), f)
            counter += 1

## Create session

In [168]:
sess = Session(sess_path)

avoid = [
#     'KinectDepth',
#     'KinectInfrared',
#     'KinectBodyIndex',
#     'KinectColor',
#     'KinectBody',
#     'KinectFaceVertices',
#     'GazeEstimation',
#     'WebCamera',
#     'InfraredCamera',
#     'Markers'
]

sess.remove_devices(*avoid)

### analyse snapshot

In [169]:
class PersonCalibration:
    
    default_parameters = np.array([0.0, 0.0, avg_eye_radius, avg_eye_radius/2, -0.03,  0.03, 0.05])
    
    def __init__(self, name='person', parameters=None):
        
        self.name = name
        
        # person parameters
        self.parameters = np.array(parameters) if parameters else self.default_parameters

    @property
    def parameters(self):
        self._parameters = None
    
    @parameters.getter
    def parameters(self):
        return self._parameters
    
    @parameters.setter
    def parameters(self, value):
        self._parameters = value
        
#         print(f'Set parameters to: {value}')
        
        # person parameters links
        self.visual_angles = self.parameters[:2].reshape(1, 2)
        self.eye_radius = self.parameters[2]
        self.cornea_radius = self.parameters[3]
        self.eye_center_head = self.parameters[4:7].reshape(1, 3)
        
    def calculate_eye_center(self, rotation, translation):
        """
        rotation (1, 3)
        translation (1, 3)

        parameters
        eye_center_head (1, 3)
        """
        # (Rodrigues((1, 3)) -> (3, 3))
        rotation_matrix = cv2.Rodrigues(rotation)[0]

        # ((3, 3) @ (3, 1) + (1, 3).T).T -> (1, 3)
        return (rotation_matrix @ self.eye_center_head.T + translation.T).T
    
    def calculate_optical_axis(self, pupil_center, eye_center):
        # ((1, 3) - (1, 3)) / scalar -> optical axis (1, 3)
        return (pupil_center - eye_center) / np.linalg.norm(pupil_center - eye_center)
    
    def calculate_cornea_center(self, eye_center, optical_axis):
        # (1, 3) + scalar * (1, 3) -> (1, 3)
        return eye_center + self.cornea_radius * optical_axis
    
    def calculate_visual_axis(self, optical_axis):
        # (((1, 3) -> (1, 2)) + (1, 2)) -> (1, 3)
        visual_axis = angles2vecs(vecs2angles(optical_axis) + self.visual_angles)
        
        # norm vector
        return visual_axis / np.linalg.norm(visual_axis)
    
    def calculate_eye_geometry(self, pupil_center, rotation, translation):
        
        eye_center = self.calculate_eye_center(rotation, translation)
        optical_axis = self.calculate_optical_axis(pupil_center, eye_center)
        cornea_center = self.calculate_cornea_center(eye_center, optical_axis)
        visual_axis = self.calculate_visual_axis(optical_axis)
        
        return cornea_center, visual_axis
    
    def gaze_direction(self, pupil_center, rotation, translation, coeff=3):
        """
        pupil_center (1, 3)
        rotation (1, 3)
        translation (1, 3)

        parameters
        eye_center_head (1, 3)
        cornea_radius scalar
        """
        cornea_center, visual_axis = self.calculate_eye_geometry(pupil_center, rotation, translation)
        
        return np.array([cornea_center.flatten(), (cornea_center + coeff * visual_axis).flatten()])
    
    def calibrate(self, points, pupil_centers, rotations, translations, alpha=1e-2):
        
        arr = np.column_stack((points, pupil_centers, rotations, translations))

        def cost(parameters):
            
            self.parameters = parameters
            
            loss = 0
            
            for s in arr:
                
                point = s[:3].reshape(1, 3)
                pupil_center, rotation, translation = s[3:6].reshape(1, 3), s[6:9].reshape(1, 3), s[9:12].reshape(1, 3)
                
                cornea_center, visual_axis = self.calculate_eye_geometry(pupil_center, rotation, translation)
                
                gaze_vector_true = point - cornea_center
                gaze_vector_true = gaze_vector_true / np.linalg.norm(gaze_vector_true)
                
                loss += np.sum((gaze_vector_true - visual_axis) ** 2) + alpha * np.linalg.norm(self.eye_center_head) ** 2
            
            return loss
        
        constraints = [
            # eye radius
            {'type': 'ineq', 'fun': lambda x: avg_eye_radius * 0.7 - x[2]},
            {'type': 'ineq', 'fun': lambda x: x[2] - avg_eye_radius * 1.3},
            
            # cornea radius
            {'type': 'ineq', 'fun': lambda x: x[2] - x[3]},
            {'type': 'ineq', 'fun': lambda x: x[3] - x[2] / 4},
            
            # visual axis angles
            {'type': 'ineq', 'fun': lambda x: x[0] - np.deg2rad(5)},
            {'type': 'ineq', 'fun': lambda x: np.deg2rad(5) - x[0]},
            {'type': 'ineq', 'fun': lambda x: x[1] - np.deg2rad(5)},
            {'type': 'ineq', 'fun': lambda x: np.deg2rad(5) - x[1]},
            
            # eye center head
            {'type': 'ineq', 'fun': lambda x: abs(x[4]) - 0.05},
            {'type': 'ineq', 'fun': lambda x: x[5] - 0.05},
            {'type': 'ineq', 'fun': lambda x: x[6] - 0.07},
            {'type': 'ineq', 'fun': lambda x: 0.05 - abs(x[4])},
            {'type': 'ineq', 'fun': lambda x: 0.05 - x[5]},
            {'type': 'ineq', 'fun': lambda x: 0.07 - x[6]},
            
        ]
        minimized = minimize(cost, x0=self.parameters, constraints=constraints, method='SLSQP')
        
        return minimized
        

In [170]:
img = np.copy(snapshot.InfraredCamera[:, ::-1, ::-1])
img = (img / img.max() * 255).astype('uint8')

pts_2d = InfraredCamera.project_vectors(face_points[list(map(int, HDFace.values()))]).astype('float64').astype(int)
pupil_test = InfraredCamera.project_vectors(pupil_3d.astype('float64')).astype(int)
nose_2d = InfraredCamera.project_vectors(face_points[int(HDFace.NoseTip)].reshape(1, 3)).astype(int)

for pt_2d in pts_2d:
    cv2.circle(img, tuple(pt_2d), 1, (255, 0, 0), -1)
cv2.circle(img, tuple(pupil_test[0].astype(int)), 1, (0, 0, 255), -1)
cv2.circle(img, tuple(pupil_center_2d[0].astype(int)), 1, (0, 255, 0))
cv2.circle(img, tuple(nose_2d[0].astype(int)), 3, (0, 0, 255), )


Image.fromarray(img)

NameError: name 'pupil_3d' is not defined

In [171]:
def quaternion_to_rotation(quaternion):
    """
    Converts angle-axis to quaternion
    :param quaternion: dict {'X': , 'Y': , 'Z': , 'W': }
    :return: angle-axis rotation vector
    """
    if isinstance(quaternion, dict):
        quaternion = dict_to_np(quaternion).flatten()
        
    assert quaternion.ndim == 1
    assert quaternion.shape == (4,)
    
    t = np.sqrt(1 - quaternion[-1] ** 2)
    
    if t:
        return (quaternion[:3] / t)
    else:
        return np.zeros((3,))

In [172]:
snapshot = next(sess.snapshots_iterate(40))
snapshot.KinectBody['0']['Joints']['Head']['Position']
q = snapshot.KinectFace[0]['FaceRotationQuaternion']
q, quaternion_to_rotation(q)

({'X': 0.104577, 'Y': 0.032921, 'Z': -0.012995, 'W': 0.993887},
 array([ 0.947225,  0.298188, -0.117707]))

In [193]:
markers = []
pupil_centers = []
rotations, translations = [], []
gazes_true = []
gazes_true_person = []
face_points_array = []

pipe = make_pipeline(PolynomialFeatures(degree=9), Ridge(1e-9))

frames = []

flip_array = np.array([-1, -1, 1])

for snapshot in sess.snapshots_iterate(40, verbose=True):
    
    # get face points
    face_points = np.array([dict_to_np(snapshot.KinectFaceVertices[idx]).flatten() for idx in hd_all])
    face_points = face_points * flip_array
    
    # get eye circle points
    EyeLeftCircle = InfraredCamera.vectors_to_self(face_points[EyeLeftCircleIdx])
    
    # get pupil center on 2D image
    pupil_2d = dict_to_np(snapshot.ManualPupils['pupilCenterLeft'])
    
    # get roi points of eye
    roi = np.array([dict_to_np(roi_pt)[0] for roi_pt in snapshot.GazeEstimation['eyeRoiLeft']])
    
    # flip x-coordinate of 2D coordinates because of mirrored image
    pupil_2d[0, 0] = 1296 - pupil_2d[0, 0]
    roi[:, 0] = 1296 - roi[:, 0]
    
    # calculate relative position of pupil center on 2D image
    relative_pupil_2d = 1 - (pupil_2d - roi[0]) / np.diff(roi, axis=0)
    
    # estimate eye surface
    XY = EyeLeftCircle[:, :2]
    Z = EyeLeftCircle[:, 2]
    pipe.fit(XY, Z)
    
    # get eye corners, top and mid points
    # and transit to Interested Camera coordinate system
    inner = InfraredCamera.vectors_to_self(face_points[int(HDFace.LefteyeInnercorner)].reshape(1, 3)).flatten()
    outer = InfraredCamera.vectors_to_self(face_points[int(HDFace.LefteyeOutercorner)].reshape(1, 3)).flatten()
    midtop = InfraredCamera.vectors_to_self(face_points[int(HDFace.LefteyeMidtop)].reshape(1, 3)).flatten()
    midbot = InfraredCamera.vectors_to_self(face_points[int(HDFace.LefteyeMidbottom)].reshape(1, 3)).flatten()
    
    # get roi points X and Y in 3D in Interested Camera coordinate system
    topinner = np.array([inner[0], midtop[1]])
    botouter = np.array([outer[0], midbot[1]])
    
    # get size of roi
    size_xy = np.abs(topinner - botouter)
    
    # estimate X and Y of 3D pupilcenter
    pupil_XY = topinner + relative_pupil_2d * size_xy
    
    # estimate Z of 3D pupil_center
    pupil_Z = pipe.predict(pupil_XY)
    pupil_XYZ = np.column_stack((pupil_XY, pupil_Z))
    
    # return pupil_center to origin coordinate system
    pupil_center = InfraredCamera.vectors_to_origin(pupil_XYZ)
    
    # get translation and rotation of head
    translation = dict_to_np(snapshot.KinectBody['0']['Joints']['Head']['Position']).flatten() * flip_array
    rotation = quaternion_to_rotation(snapshot.KinectFace[0]['FaceRotationQuaternion']) # * flip_array
    
    # get marker
    marker = dict_to_np(snapshot.Markers) * flip_array
    
    # create person device
    person = Device(name='person', translation=translation.tolist(), rotation=rotation.tolist())
    
    # calculate true gaze
    gaze_true = marker - pupil_center
    gaze_true = gaze_true / np.linalg.norm(gaze_true)
    
    # calculate true gaze in person coordinate system
    gaze_true_person = person.vectors_to_self(marker) - person.vectors_to_self(pupil_center)
    gaze_true_person = gaze_true_person / np.linalg.norm(gaze_true_person)
    
    # append data
    pupil_centers.append(pupil_center.flatten())
    rotations.append(rotation)
    translations.append(translation)
    markers.append(marker.flatten())
    gazes_true.append(gaze_true.flatten())
    gazes_true_person.append(gaze_true_person.flatten())
    face_points_array.append(person.vectors_to_self(face_points))
    
    # face vector for visualization
    faceGaze = np.array([translation.flatten(), (translation - 4 * rotation).flatten()])
    
pupil_centers = np.array(pupil_centers)
rotations = np.array(rotations)
translations = np.array(translations)
markers = np.array(markers)
gazes_true = np.array(gazes_true)
gazes_true_person = np.array(gazes_true_person)
face_points_array = np.array(face_points_array)

100%|██████████| 40/40 [00:07<00:00,  5.64it/s]


In [178]:
def create_trace_3d(data, **kwargs):
    
    assert data.ndim == 2
    assert data.shape[1] == 3
    
    x, y, z = data.T
    return go.Scatter3d(x=x, y=y, z=z, **kwargs)

marker_style={
    'mode': 'markers',
    'marker': dict(size=3, line=dict(color='rgba(217, 217, 217, 0.14)', width=0.5), opacity=0.8)
}

In [179]:
i = 9

traces = [create_trace_3d(data.reshape(-1, 3), **marker_style) for data in [np.array([[0, 0, 0]]), pupil_centers[i], face_points_array[i]]]

layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [192]:
np.diff(face_points_array, axis=0).mean(axis=0).mean(axis=0)

array([3.916399e-04, 2.601291e-04, 8.654017e-05])

In [181]:
np.diff(face_points_array, axis=0).mean(axis=0).mean(axis=0)

array([3.916399e-04, 2.601291e-04, 8.654017e-05])

In [191]:
p = PersonCalibration()

p.calibrate(markers, pupil_centers, rotations, translations, alpha=0)

     fun: 7.535450159994196
     jac: array([ -18.728806,   20.128433,    0.      ,    3.912129, -104.700407,
         35.517195, -129.365791])
 message: 'Optimization terminated successfully.'
    nfev: 42
     nit: 8
    njev: 4
  status: 0
 success: True
       x: array([ 8.726646e-02,  4.564933e-10,  7.462749e-03,  1.865685e-03,
       -3.000000e-02,  3.000000e-02,  7.000000e-02])

In [190]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

X = np.column_stack((pupil_centers, translations, rotations))
Y = vecs2angles(gazes_true)

pipe = make_pipeline(PolynomialFeatures(5), Ridge(1e-3))

kf = KFold(40)

for train_index, test_index in kf.split(X):
    
    # fit
    pipe.fit(X[train_index], Y[train_index])
    
    print('===========')
    print('score: ', pipe.score(X, Y))
    
    # test
    Y_pred = pipe.predict(X[test_index])
    print('train point: ', np.rad2deg(pipe.predict(X[train_index]) - Y[train_index]).mean(axis=0))
    print('test point: ', test_index, np.rad2deg(Y_pred - Y[test_index]))

score:  0.8978309223391662
train point:  [-6.855826e-13  1.066725e-12]
test point:  [0] [[-21.116293   4.046219]]
score:  0.9143591946791323
train point:  [ 2.492365e-13 -2.386239e-13]
test point:  [1] [[-0.598704  8.281738]]
score:  0.9157020188535073
train point:  [1.794075e-12 1.013161e-12]
test point:  [2] [[2.419248 7.560866]]
score:  0.904577004564373
train point:  [6.548836e-13 1.802820e-12]
test point:  [3] [[15.166697  3.077972]]
score:  0.8991543876648641
train point:  [-1.992298e-12  2.801406e-13]
test point:  [4] [[-10.033788 -13.447045]]
score:  0.9173830258415121
train point:  [-4.600309e-15  5.430186e-13]
test point:  [5] [[-5.349586 -2.918549]]
score:  0.9172554210817508
train point:  [-4.946471e-14  7.654732e-13]
test point:  [6] [[-1.439169 -5.179692]]
score:  0.907861152739851
train point:  [ 4.053282e-13 -1.227804e-12]
test point:  [7] [[15.267747 -4.341692]]
score:  0.8972245932531644
train point:  [-5.506706e-13 -4.782271e-13]
test point:  [8] [[ -5.330162 -16.260